In [1]:
!nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address     Load        Tokens  Owns (effective)  Host ID                               Rack 
UN  172.21.0.4  185.41 KiB  16      100.0%            ab484865-e62b-4d27-9fae-d22e5005ea4e  rack1
UN  172.21.0.2  243.52 KiB  16      100.0%            80bafe9b-0c72-4900-983b-8fd6a5d91e31  rack1



In [2]:
from cassandra.cluster import Cluster
cluster = Cluster(['p6-db-1', 'p6-db-2', 'p6-db-3'])
cass = cluster.connect()

In [3]:
cass.execute("drop keyspace if exists weather")
cass.execute("""create keyspace weather with replication = {
    'class' : 'SimpleStrategy',
    'replication_factor' : 3
    };
""")
cass.execute("use weather")
cass.execute("""
    create type station_record(
        tmin INT,
        tmax INT
    )
""")
cass.execute("""
    create table stations(
        id TEXT,
        name TEXT STATIC,
        date DATE,
        record WEATHER.STATION_RECORD,
        PRIMARY KEY ((id), date)
    ) WITH CLUSTERING ORDER BY (date ASC)
""")


In [4]:
#q1
cass.execute("describe table weather.stations").one().create_statement

"CREATE TABLE weather.stations (\n    id text,\n    date date,\n    name text static,\n    record station_record,\n    PRIMARY KEY (id, date)\n) WITH CLUSTERING ORDER BY (date ASC)\n    AND additional_write_policy = '99p'\n    AND bloom_filter_fp_chance = 0.01\n    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}\n    AND cdc = false\n    AND comment = ''\n    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}\n    AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}\n    AND memtable = 'default'\n    AND crc_check_chance = 1.0\n    AND default_time_to_live = 0\n    AND extensions = {}\n    AND gc_grace_seconds = 864000\n    AND max_index_interval = 2048\n    AND memtable_flush_period_in_ms = 0\n    AND min_index_interval = 128\n    AND read_repair = 'BLOCKING'\n    AND speculative_retry = '99p';"

In [5]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
         .appName("p6")
         .config('spark.jars.packages', 'com.datastax.spark:spark-cassandra-connector_2.12:3.4.0')
         .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions")
         .getOrCreate())

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-70d5faf2-510f-45fc-b257-6450fe24c39f;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.4.0 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.4.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found com.github.stephenc.jcip#jcip-annotations;1.0-1 in central
	found com.gith

In [9]:
from pyspark.sql.functions import col, expr
import pandas as pd
df = spark.read.text("ghcnd-stations.txt")
ID_df = df.withColumn("ID", expr("substring(value, 0, 11)"))
STATE_df = df.withColumn("STATE", expr("substring(value, 39, 2)"))
NAME_df = df.withColumn("NAME", expr("substring(value, 42, 30)"))
df2 = ID_df.join(STATE_df, ['value'], how='inner')
df2 = df2.join(NAME_df, ['value'], how='inner')
filtered_df = df2.where(col("STATE") == "WI")

df_insert = cass.prepare("""
    INSERT INTO stations (id, name)
    VALUES (?, ?)
""")

filtered_df = filtered_df.toPandas()
for index, row in filtered_df.iterrows():
    cass.execute(df_insert, (filtered_df["ID"][index], filtered_df["NAME"][index]))

,id,date,name,record
0,USC00479053,None,W BEND FIRE STN #2,None
1,USC00476398,None,PARK FALLS DNR HQ,None
2,USC00470268,None,APPOLONIA,None
3,USC00474110,None,JUNEAU,None
4,USC00475525,None,MINONG 5 WSW,None
...,...,...,...,...
1308,USC00479050,None,WEST BEND,None
1309,US1WIPC0004,None,RIVER FALLS 3.0 SE,None
1310,US1WIWK0016,None,MUSKEGO 1.0 W,None
1311,US1WIWK0086,None,WALES 0.4 NW,None


In [53]:
#q2
df = pd.DataFrame(cass.execute("""
    SELECT name from weather.stations
    WHERE id='USW00014837'
"""))

df["name"][0]

'MADISON DANE CO RGNL AP       '

In [50]:
#q3
pd.DataFrame(cass.execute("""
    SELECT * from weather.stations
    WHERE id='USC00470273'
"""))


,id,date,name,record
0,USC00470273,None,UW ARBORETUM - MADISON,None


In [54]:
#q4
!nodetool ring weather
# import subprocess
# sp = subprocess.check_output("nodetool ring weather", stderr=subprocess.STDOUT, shell=True)
# sp


Datacenter: datacenter1
Address          Rack        Status State   Load            Owns                Token                                       
                                                                                9045517454624181859                         
172.21.0.2       rack1       Up     Normal  240.08 KiB      100.00%             -8760787059843916800                        
172.21.0.4       rack1       Up     Normal  265.83 KiB      100.00%             -8142806027941716722                        
172.21.0.2       rack1       Up     Normal  240.08 KiB      100.00%             -7689286443503883987                        
172.21.0.4       rack1       Up     Normal  265.83 KiB      100.00%             -7078465038168653325                        
172.21.0.2       rack1       Up     Normal  240.08 KiB      100.00%             -6671435038761456971                        
172.21.0.2       rack1       Up     Normal  240.08 KiB      100.00%             -5853763106149106177